# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### Nordvig

In [9]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('data/big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]).union(known(edits1(word))) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Spell corrector

In [10]:
bigrams = open('data/bigrams.txt', 'r', encoding='latin-1')
fivegrams = open('data/fivegrams.txt', 'r')

two_grams = {}
for line in bigrams:
    words = line.strip().split('\t')
    
    two_grams[tuple(words[1:])] = int(words[0])


three_grams = {}
four_grams = {}
five_grams = {}
for line in fivegrams:
    words = line.strip().split('\t')

    freq = int(words[0])
    
    five_grams[tuple(words[1:])] = freq
    
    for gram in [words[1:-1], words[2:]]:
        if tuple(gram) in four_grams: four_grams[tuple(gram)] += freq
        else: four_grams[tuple(gram)] = freq
    
    for gram in [words[1:4], words[2:5], words[3:]]:
        if tuple(gram) in three_grams: three_grams[tuple(gram)] += freq
        else: three_grams[tuple(gram)] = freq


In [11]:
from collections import defaultdict

grams = [None, None, two_grams, three_grams, four_grams, five_grams]

def find_word_in_context_by_ngrams(ngrams, word, context):

    '''
    Find word in context by ngrams

    Parameters
    ----------
    ngrams : list
        List of n-grams
    word : str
        Word
    context : list
        Context of the word

    Returns
    -------
    list
        List of contexts
    '''
    
    # len of context can at most be 5
    context = context[-5:]
    
    context.append(word)
    to_return = defaultdict(list)
    for ind, ngram in enumerate(ngrams):
        if not ngram: continue

        # sliced part of the context based on current ngram
        context_slice = tuple(context[-ind:])
        if context_slice in ngram.keys():
            # add current slice to the list of contexts
            if not to_return[context_slice]: 
                to_return[context_slice] = ngram[context_slice]
            else: 
                to_return[context_slice] += ngram[context_slice]
    
    return to_return

In [12]:
find_word_in_context_by_ngrams(grams, 'in', ['a', 'babe'])


defaultdict(list, {('babe', 'in'): 73, ('a', 'babe', 'in'): 16})

In [18]:
def get_best_word(list_of_contexts):

    '''
    Get the best word from the contexts of all candidates

    Parameters
    ----------
    list_of_contexts : list
        List of contexts
    
    Returns
    -------
    str
        Best word out of all contexts
    '''

    max_value = 0 # number of occurrences of the context
    best_word = None # best word out of all contexts
    max_gram_len = 0 # length of the best ngram
    for context in list_of_contexts:
        if len(context) == 0: continue 
        
        for gram in context:
            # take the largest context possible
            if max_gram_len <= len(gram):
                # if len of the current ngram is larger than definately update the max value
                if max_gram_len < len(gram):
                    max_value = context[gram]
                # else update the max value only if it is larger
                else:
                    if max_value < context[gram]:
                        max_value = context[gram]

                max_gram_len = len(gram)
                
                # and also choose the best word in that context
                best_word = gram[-1]

    return best_word

def correct_word(grams, word, context):

    '''
    Make a correction of given misspelled word.

    Parameters
    ----------
    grams : list
        List of n-grams
    word : str
        Misspelled word
    context : list
        Context of the word

    Returns
    -------
    str
        Corrected word
    '''

    # create a list of candidates
    cands = candidates(word)

    # get the contexts of all candidates
    list_of_contexts = [find_word_in_context_by_ngrams(grams, c, context) for c in cands]

    flag = False
    for c in list_of_contexts:
        if len(list(c.items())) > 0:
            flag = True
            break
    
    # just return the best candidate if there is no context for any candidate
    if not flag: 
        return max(cands, key=P)

    return get_best_word(list_of_contexts)


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [17]:
import random
from tqdm import tqdm

def spoil_word(word):
    # use nordvig edit1 to spoil the word
    spoiled_word = list(edits1(word))[0]
    return spoiled_word

file = open('data/english.txt', 'r')

counter_mine = 0
counter_nordvig = 0
file_len = 0
for line in file:

    spoiled_word_index = random.randint(1, len(line.split()) - 1)
    spoiled_word = spoil_word(line.split()[spoiled_word_index])

    context = line.split()[0:spoiled_word_index]
    
    corrected_word = correct_word(grams, spoiled_word, context)
    if corrected_word == line.split()[spoiled_word_index]:
        counter_mine += 1
    
    nordvig_corrected_word = correction(spoiled_word)
    if nordvig_corrected_word == line.split()[spoiled_word_index]:
        counter_nordvig += 1
    
    file_len += 1

print('My accuracy', counter_mine/file_len)
print('Nordvig accuracy', counter_nordvig/file_len)


My accuracy 0.6975
Nordvig accuracy 0.69125
